This file contains the program used to hyperparameter tune our hyperparameters with pytorch:
Learning Rate
Number of Epochs
Batch Size


In [24]:
from functions_text_model import *
import os

In [25]:
current_directory = os.getcwd()
print("Current directory:", current_directory)

contents = os.listdir(current_directory)
print("Contents of current directory:", contents)

#### Change the Dataset here when neccessary!
json_file_path = os.path.join(current_directory, 'data', 'mixed_data.json')
print(json_file_path)

Current directory: /Users/regulafrey/Desktop/Deep_Learning/Deep_Learning_Project/Inclusiveness-in-Sarcasm-Detection
Contents of current directory: ['text-data-preparation.ipynb', 'text-model-training.ipynb', 'requirements.txt', 'functions_text_model.py', 'audio_features.p', 'text-model-training_context.ipynb', 'functions_audio_model.py', 'results.json', 'video-model-training.ipynb', 'results_tinybert.json', 'z-old-text-model.ipynb', '__pycache__', 'README.md', 'Tryouts', '.gitignore', 'z-old-text-model-BERT-training.ipynb', 'audio_model-HuBERT.ipynb', 'hyperparameter_tuning.ipynb', 'results_tinybert_linear_added.json', 'text-model-tinybert-fallback.pth', '.git', 'data', 'hyperparameter_tuning_text_context.ipynb', 'functions_video_model.py']
/Users/regulafrey/Desktop/Deep_Learning/Deep_Learning_Project/Inclusiveness-in-Sarcasm-Detection/data/mixed_data.json


In [30]:
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
import json
from transformers import BertForSequenceClassification, BertTokenizer
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.utils.data import random_split
from sklearn.model_selection import ParameterGrid


# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Function to encode the text
def encode_text(text):
    encoded_dict = tokenizer.encode_plus(
                        text,                      # Input text
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences
                        truncation = True,
                        padding = 'max_length',
                        return_attention_mask = True,   # Construct attention masks
                        return_tensors = 'pt',     # Return pytorch tensors
                   )
    return encoded_dict['input_ids'], encoded_dict['attention_mask']

# PyTorch Dataset
class SarcasmDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        utterance = item['utterance']
        context = item['context']
        utterance_and_context = ' '.join([sentence for sentence in context] + [utterance]) # Combining the utterance and its context into one string.
        sarcasm = int(item['sarcasm'])
        input_ids, attention_mask = encode_text(utterance_and_context)
        return input_ids.flatten(), attention_mask.flatten(), sarcasm
    
# Create the DataLoader
# Load the data from the JSON file
with open(json_file_path) as f:
    data = json.load(f)

# Convert the data to a list of dictionaries
data = list(data.values())

dataset = SarcasmDataset(data)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [31]:
torch.manual_seed(42)
    
# Set device
device = torch.device("cpu")

# Define the hyperparameters to tune
param_grid = {
    'lr': [1e-3, 1e-4, 1e-5],
    'num_epochs': [7],
    'batch_size': [8, 16, 32, 64]
}


# Create a parameter grid
grid = ParameterGrid(param_grid)

# Initialize a list to store the results
results = []

# For each combination of hyperparameters
for params in grid:
    # Create a new model
    model = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased",
        num_labels = 2,
        output_attentions = False,
        output_hidden_states = False,
    )

    for parameter in model.bert.parameters():
        parameter.requires_grad = False

    model.to(device)

    # Create a new optimizer with the current learning rate
    optimizer = AdamW(model.classifier.parameters(), lr=params['lr'])
    # Create the optimizer  
    criterion = nn.CrossEntropyLoss()

    metrics = {'ACC': acc, 'F1-weighted': f1}
    # Define the size of the training set and the test set
    train_size = int(0.8 * len(dataset))  # 80% of the data for training
    test_size = len(dataset) - train_size  # 20% of the data for testing

    # Split the dataset
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    # Create the DataLoaders
    train_dataloader = DataLoader(train_dataset, batch_size=params['batch_size'])
    test_dataloader = DataLoader(test_dataset, batch_size=params['batch_size'])


    # Train and evaluate the model for the current number of epochs
    for epoch in range(params['num_epochs']):
        print(f"Epoch {epoch + 1}")
        print('learning rate:', params['lr'], 'batch size:', params['batch_size'], 'num_epochs:', params['num_epochs'])
        train_loss, train_metrics = train_epoch(model, optimizer, criterion, metrics, train_dataloader, device)
        eval_loss, eval_metrics = evaluate(model, criterion, metrics, test_dataloader, device)

    # Store the results
    results.append({
        'lr': params['lr'],
        'batch_size': params['batch_size'],
        'num_epochs': params['num_epochs'],
        'eval_loss': eval_loss,
        'eval_metrics': eval_metrics
    })

# Save the results to a JSON file
#### Change Filename here when needed!
with open('results_hyperparameter_text_context_mixed.json', 'w') as f:
    json.dump(results, f)

print(results)

train Loss: 0.7033,  ACC: 0.5094, F1-weighted: 0.4456


100%|██████████| 2/2 [00:05<00:00,  2.98s/it]

eval Loss: 0.7008,  ACC: 0.4722, F1-weighted: 0.3448
[{'lr': 0.001, 'batch_size': 8, 'num_epochs': 7, 'eval_loss': 0.6381482444703579, 'eval_metrics': {'ACC': 0.59375, 'F1-weighted': 0.5867649711399712}}, {'lr': 0.0001, 'batch_size': 8, 'num_epochs': 7, 'eval_loss': 0.6864803209900856, 'eval_metrics': {'ACC': 0.5279017857142857, 'F1-weighted': 0.4649926462426462}}, {'lr': 1e-05, 'batch_size': 8, 'num_epochs': 7, 'eval_loss': 0.6904398947954178, 'eval_metrics': {'ACC': 0.4966517857142857, 'F1-weighted': 0.4274825868575869}}, {'lr': 0.001, 'batch_size': 16, 'num_epochs': 7, 'eval_loss': 0.6950617507100105, 'eval_metrics': {'ACC': 0.54375, 'F1-weighted': 0.43466430610452356}}, {'lr': 0.0001, 'batch_size': 16, 'num_epochs': 7, 'eval_loss': 0.700546570122242, 'eval_metrics': {'ACC': 0.45, 'F1-weighted': 0.3761401348143339}}, {'lr': 1e-05, 'batch_size': 16, 'num_epochs': 7, 'eval_loss': 0.7107417955994606, 'eval_metrics': {'ACC': 0.44114583333333335, 'F1-weighted': 0.33333803877282137}}, {'l